In [ ]:
from pathlib import Path
import datetime

import requests
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, Limiter, RequestRate
from tqdm import tqdm

import stock


class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass


session = CachedLimiterSession(
    limiter=Limiter(RequestRate(10, Duration.SECOND)),  # max 2 requests per 1 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("gmo.cache"),
)

In [ ]:
start_date = datetime.date(2018, 9, 5)
end_date = datetime.date.today() - datetime.timedelta(days=1)

ticker_list = [
    "BTC",
    "ETH",
    "BCH",
    "LTC",
    "XRP",
    "XEM",
    "XLM",
    "BAT",
    "XTZ",
    "QTUM",
    "ENJ",
    "DOT",
    "ATOM",
    "MKR",
    "DAI",
    "XYM",
    "MONA",
    "FCR",
    "ADA",
    "LINK",
    "DOGE",
    "SOL",
    "ASTR",
    "BTC_JPY",
    "ETH_JPY",
    "BCH_JPY",
    "LTC_JPY",
    "XRP_JPY",
    "DOT_JPY",
    "ATOM_JPY",
    "ADA_JPY",
    "LINK_JPY",
    "DOGE_JPY",
    "SOL_JPY",
]
base_url = "https://api.coin.z.com/data/trades/{ticker}/{yyyy}/{mm:02d}/{yyyymmdd}_{ticker}.csv.gz"
output_root_dir = stock.PROJECT_ROOT / "data" / "tick_data"

In [ ]:
for ticker in ticker_list:
    date = start_date
    while date <= end_date:
        url = base_url.format(ticker=ticker, yyyy=date.year, mm=date.month, yyyymmdd=date.strftime("%Y%m%d"))
        output_dir = output_root_dir / date.strftime("%Y%m%d")
        output_dir.mkdir(exist_ok=True)
        output_path = output_dir / Path(url).name

        response = requests.get(url)
        if response.status_code == 200:
            with output_path.open("wb") as f:
                f.write(response.content)
        else:
            print(f"Failed to download {url}")
        date += datetime.timedelta(days=1)

In [ ]:
output_dir = output_root_dir / date.strftime("%Y%m%d")
output_dir.mkdir(exist_ok=True)
output_path = output_dir / Path(url).name

In [ ]:
with output_path.open("wb") as f:
    f.write(response.content)

In [ ]:
response.status_code